In [1]:
import os

import torch
import torchvision
import torch.nn as nn
import torch.functional as F
import torchvision.models as models
import torchvision.transforms as transforms

# define Model

In [2]:
resnet_152 = models.resnet152(pretrained=True, progress=True)

## define device

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

resnet_152.to(device)

if device == torch.device('cuda') :
    print("device : 'cuda'")
else :
    print("device : 'cpu'")

device : 'cuda'


# Transform & DataLoader

In [4]:
transform = transforms.Compose(
    [transforms.ToTensor(),

    # channel number 만큼 수 전달.
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


batch_size = 12


data_path = '../ResNet_utils/data'

trainset = torchvision.datasets.CIFAR10(
    root=data_path, train=True,
    download=True, transform=transform
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size,
    shuffle=True, num_workers=2
)


testset = torchvision.datasets.CIFAR10(
    root=data_path, train=False,
    download=True, transform=transform
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=batch_size,
    shuffle=True, num_workers=2
)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
len(trainset)

50000

# define Optimizer

In [6]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet_152.parameters(), lr=0.001)

# Training (with model CheckPoint)

In [12]:
model_path = '../ResNet_utils/trained_models/checkpoints/'

for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받기
        inputs, labels = data[0].to(device), data[1].to(device)

        # Gradient parameter to Zeros
        optimizer.zero_grad()

        # foward/back propagation + optimization
        outputs = resnet_152(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print info
        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 2000}")

            model_path_ = os.path.join(model_path, f"ResNet152_{epoch}_{i+1}_{running_loss/2000}.pth")
            torch.save({
                'epoch' : epoch,
                'model_state_dict' : resnet_152.state_dict(),
                'optimizer_state_dict' : optimizer.state_dict(),
                'loss' : running_loss / 2000
            }, model_path_)
            
            running_loss = 0.0
        
        torch.cuda.empty_cache()

print('Finished Training')

[1, 2000] loss: 1.574141943693161
[1, 4000] loss: 1.3726720532774925
[2, 2000] loss: 1.2510065106451511
[2, 4000] loss: 1.1630401315540075
Finished Training


# Saving & Loading Model for Inference

## Save/Load ***state_dict*** (Recommended)

### Save:

In [ ]:
torch.save(model.state_dict, model_path)

### Load:

In [ ]:
resnet_152 = models.resnet152()
resnet_152.load_state_dict(torch.load(model_path))

## Save/Load Entire Model

### Save:

In [ ]:
torch.save(resnet_152, model_path) # => to .pt or .pth

### Load:

In [ ]:
resnet_152 = torch.load(model_path)

## Saving & Loading a General Checkpoint for Inference and/or Resuming Training

### Save:

In [ ]:
torch.save({
    'epoch' : epoch,
    'model_state_dict' : resnet_152.state_dict(),
    'optimizer_state_dict' : optimizer.state_dict(),
    'loss' : loss
}, model_path)

### Load:

In [ ]:
resnet_152 = models.resnet152()
optimizer = optim.SGD()

checkpoint = torch.load(model_path)
resnet_152.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

## Saving Multiple Models in One File

### Save:

In [ ]:
torch.save({
    'modelA_state_dict' : resnet_152A.state_dict(),
    'modelB_state_dict' : resnet_152B.state_dict(),
    'optimizerA_state_dict' : optimizerA.state_dict(),
    'optimizerB_state_dict' : optimizerB.state_dict()
}, model_path)

### Load:

In [ ]:
resnet_152A = models.resnet152()
resnet_152B = models.resnet152()
optimizerA = optim.SGD()
optimizerB = optim.Adagrad()

checkpoint = torch.load(model_path)
resnet_152A.load_state_dict(chekcpoint['modelA_state_dict'])
resnet_152B.load_state_dict(checkpoint['modelB_state_dict'])
optimizerA.load_state_dict(checkpoint['optimizerA_state_dict'])
optimizerB.load_state_dict(checkpoint['optimizerB_state_dict'])

